In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Clustering/data')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


Mounted at /content/drive
/bin/bash: line 1: nvidia-smi: command not found
Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
#!pip install numpy==1.16.0
!pip install scikit-learn-extra
!pip install umap-learn
# !pip install hdbscan
!pip install git+https://github.com/scikit-learn-contrib/hdbscan.git
!pip install tslearn
!pip install geopandas
!pip install -q ipywidgets
!pip3 install ipympl
%load_ext autoreload
%autoreload 2
!pip install folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82813 sha256=f38d2faa33c91a549f4472a62664dc4e9a88424c6c36d638956cbf5cd12e308d
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55621 sha256=d7ef431ae1da977ab341d499f47b1f9273739146b52cb937fc75eddbdedcdb64
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent
  Cloning https://github.com/scikit-learn-contrib/hdbscan.git to /tmp/pip-req-build-8rwt2et7
  Running comm

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import correlate
import plotly.graph_objects as go
import statsmodels.api as sm
from scipy import stats
from scipy.stats import iqr
from itertools import combinations
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import DBSCAN
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.manifold import TSNE
from umap import UMAP
import matplotlib.image as mpimg
import hdbscan
from sklearn.cluster import AgglomerativeClustering
from tslearn.metrics import dtw,lcss
import plotly.graph_objects as go
import plotly.express as px
from tslearn.clustering import TimeSeriesKMeans
#from tslearn.clustering import silhouette_score
from sklearn.metrics import silhouette_samples,silhouette_score
import geopandas as gpd
import time
from collections import Counter
import math
from sklearn.manifold import Isomap
import pywt
import pywt.data
import plotly.subplots as sp
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from matplotlib.pyplot import get_cmap
from matplotlib.pyplot import cm
from scipy.spatial import distance
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from scipy.signal import find_peaks
from statsmodels.tsa.stattools import acf,pacf
import folium

%matplotlib inline
#%matplotlib notebook
"""
extract features and cluster feature vectors from all stations.
use closest actual station to centroid to be the representative station.
Only analyze the representative stations and NOT all.
This is a dimensionality reduction technique.

"""

In [ ]:
num_of_clusters = 66
timeslot_duration = 10
total_timelots = int(1440/timeslot_duration)
os.chdir('/content/drive/MyDrive/Clustering/data')
demand = np.load('A_21.npy',allow_pickle=True).reshape((num_of_clusters, num_of_clusters, total_timelots, timeslot_duration)).sum(axis=3)
demand = demand.sum(axis=1).T
manhat_demand = pd.DataFrame(data=demand,index=pd.date_range(start='2020-01-21 00:00:00', end='2020-01-21 23:50:00', freq='10T'),columns=[ str(i) for i in range(demand.shape[1])])
manhat_demand

In [ ]:
"""
Visualize demand on a map
"""

# housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
# s=housing["population"]/100, label="population", figsize=(10,7),
# c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
# )
# plt.legend()

#TODO: Refernce: https://github.com/fmaletski/nyc-taxi-map/tree/master

In [ ]:
"""
Exploratory data analysis: Check if n95 is different from Max and that it lies within the IQR of most stations.
"""
df = manhat_demand
# df = df.drop(columns=['week','hour','weekday'])
percentiles_95 = df.quantile(0.95)
num_series = len(df.columns)

plt.figure(figsize=(20, 20))
plt.boxplot(df, positions=np.arange(num_series), vert=False)
plt.scatter(percentiles_95, np.arange(num_series), color='red', marker='o', label='95th Percentile')
plt.yticks(np.arange(num_series), df.columns)
plt.xlabel('Value')
plt.ylabel('Series')
plt.title("Boxplot of station's Time Series Data with 95th Percentile")
plt.legend()
plt.show()


In [ ]:
manhattan_zones = pd.read_csv('manhattan_zones.csv')

zone_to_gps_dict = {}

for i in manhattan_zones.index:
    lat = manhattan_zones.iloc[i]['lat']
    lng = manhattan_zones.iloc[i]['lng']
    zone_to_gps_dict[i] = (lat,lng)


In [ ]:
# def feature_standardization(X_train):
#     # ensure that features of different scale are comparable using euclidean distance
#     X_train_Df = pd.DataFrame(X_train)
#     X_train_normalized =(X_train_Df-X_train_Df.mean())/X_train_Df.std()
#     X_train_normalized = X_train_normalized.fillna(0)
#     return X_train_normalized.to_numpy()

class feat_transformation():
    def __init__(self,):
        return
    def feature_normalization(self,X_train):
        # ensure that features of different scale are comparable using euclidean distance
        self.scaler = MinMaxScaler() #StandardScaler()
        self.scaler.fit(X_train)
        X_train_normalized = self.scaler.transform(X_train)
        return X_train_normalized

    def inv_feature_normalization(self,X_train_normalized):
        # ensure that features of different scale are comparable using euclidean distance
        X_train = self.scaler.inverse_transform(X_train_normalized)
        return X_train


In [ ]:
## Visualize the clustered time series


def plot_stations_of_clusters(labels, df,TS_type,X_train,X_train_inv_normalized,plot_type,angular_labels,plot_title,mother_wavelet,limit_level):
    unique_clusters = np.unique(labels)
    num_clusters = len(unique_clusters)
    cluster_to_stations_dict = { l: [] for l in unique_clusters}

    cols = df.columns
    station_to_index_dict = {}
    index=0
    if isinstance(cols[index], tuple): # differentiate between Dim_Red_unmet_60min_outflow_week_df DF and unmet_60min_outflow_df DF
      for stn in cols: # station to index mapping
        station_to_index_dict[stn[0]] = index
        index+=1
    else:
      for stn in cols: # station to index mapping
        station_to_index_dict[stn] = index
        index+=1

    idx=0
    if isinstance(cols[idx], tuple): # differentiate between Dim_Red_unmet_60min_outflow_week_df DF and unmet_60min_outflow_df DF
      for l in labels:
        cluster_to_stations_dict[l].append(cols[idx][0])
        idx+=1
    else:
      for l in labels:
        cluster_to_stations_dict[l].append(cols[idx])
        idx+=1

    #shapefile_data = gpd.read_file('NYC Taxi Zones (1)/taxi_zones.shp',)

    num_features = X_train.shape[1]
    all_cluster_feat_mins = np.array([])
    all_cluster_feat_maxs = np.array([])

    ################## par coords plot ##################
    if(plot_type == 'par_coords'): # parallel coordinates
        for key in cluster_to_stations_dict.keys():

          #fig0, ax0 = plt.subplots()
          #shapefile_data.plot(ax=ax0,edgecolor='black', linewidth=0.9, color='lightblue',figsize=(5,5))

          #fig = go.Figure()
          #fig1 = go.Figure()
          fig2, ax2 = plt.subplots(figsize=(30,5))
          #fig3, ax3 = plt.subplots(figsize=(60,5))
          fig4, ax4 = plt.subplots(figsize=(30,5))

          color_feature = 0  # The feature index to use for colors
          color_map = get_cmap('tab20')  # Choose a colormap

          d = {}

          feat_min_per_cluster = [1000]*num_features
          feat_max_per_cluster = [-1000]*num_features

          cluster_data = []
          cluster_stns_gps_coords = []

          for station in cluster_to_stations_dict[key]:
            #ax0.scatter(zone_to_gps_dict[int(station)][0], zone_to_gps_dict[int(station)][1], color='red', marker='.')

            cluster_stns_gps_coords.append(zone_to_gps_dict[int(station)])

            #y_arr = np.array(df[station].values).reshape(-1)
            # fig = fig.add_trace(go.Scatter(x=np.arange(len(y_arr)), y=y_arr,mode='lines+markers',name=station))
            # fig.update_layout(title='cluster: '+ str(key),xaxis_title="time (Monday 0:00 - Sunday 23:00)",
            # yaxis_title=TS_type,autosize=False,width=1800,height=400, )

            # wave_coeffs = pywt.wavedec( df[station] , mother_wavelet, level=limit_level)
            # rec_wave = pywt.waverec(wave_coeffs, mother_wavelet)
            # ym = np.median(rec_wave)
            # ax3.plot(np.linspace(0, 1., num=len(rec_wave)), rec_wave-ym)
            # #ax3.plot(df[station])

            d['r'] = X_train[station_to_index_dict[station]]
            # d['theta'] = angular_labels
            # fig1.add_trace(go.Scatterpolar(
            #     r=d['r'],
            #     theta=d['theta'],
            #     fill='toself',name=station,
            # ))

            sample = X_train[station_to_index_dict[station]]  # Get the current sample
            color = color_map(sample[color_feature])  # Get the color based on the color feature value
            ax2.plot(range(num_features), sample, color=color, alpha=0.7, label=station)

            #unnormalized features
            sample1 = X_train_inv_normalized[station_to_index_dict[station]]  # Get the current sample
            cluster_data.append(sample1)

            # histogram and kernel density estimate
            sns.histplot(data=df[station].values, kde=False, stat="count", ax=ax4)
            ax5 = ax4.twinx()
            sns.kdeplot(data=df[station].values, ax=ax5)

            for f in range(num_features):
                feat_min_per_cluster[f] = min(feat_min_per_cluster[f],sample[f])
                feat_max_per_cluster[f] = max(feat_max_per_cluster[f],sample[f])

          all_cluster_feat_mins = np.append(all_cluster_feat_mins,np.array(feat_min_per_cluster))
          all_cluster_feat_maxs = np.append(all_cluster_feat_maxs,np.array(feat_max_per_cluster))

          cluster_data = np.array(cluster_data)

          # fig1.update_layout(title='cluster: '+ str(key),
          #     polar=dict(
          #         radialaxis=dict(visible=True,))#range=[-2, 7]))
          #     ,height=800,width=800,
          #     showlegend=True
          # )
          # fig1.show()

          # Create a map centered on the coordinates of interest
          map_center = [40.7831, -73.9712]  # Manhattan coordinates (latitude, longitude)
          map_zoom = 13  # Adjust the zoom level as needed
          my_map = folium.Map(location=map_center, zoom_start=map_zoom,width='40%', height='40%')
          for coord in cluster_stns_gps_coords:
              folium.Marker(location=coord).add_to(my_map)
          # Display the map
          display(my_map)

          # ax0.set_xlabel('Longitude')
          # ax0.set_ylabel('Latitude')
          # ax0.set_title('manhattan zones of cluster: '+ str(key))
          # fig0.show()

          #fig.show()

          ax2.set_xticks(range(num_features))
          ax2.set_xticklabels([i for i in angular_labels])
          ax2.set_ylabel('Normalized Value')
          #ax2.set_ylim(0, 1)  # Modify as needed
          ax2.set_title(plot_title+', cluster: '+str(key)+' has '+str(len(cluster_to_stations_dict[key]))+' stations')
          ax2.grid(True, linestyle='--')
          #fig2.tight_layout()
          fig2.show()

          # #ax3.set_xticklabels([i for i in unmet_60min_outflow_df.index])
          # plt.xticks(rotation=90)
          # ax3.set_ylabel('Low frequency, '+'level: '+ str(limit_level) + ', '+ mother_wavelet +' Wavelet reconstruction')
          # #ax3.set_ylim(0, 1)  # Modify as needed
          # ax3.set_title(plot_title+', cluster: '+str(key)+' has '+str(len(cluster_to_stations_dict[key]))+' stations')
          # #ax3.legend()
          # ax3.grid(True, linestyle='--')
          # #fig3.tight_layout()
          # fig3.show()


          cluster_df = pd.DataFrame(cluster_data, columns=angular_labels)
          cluster_df['sample'] = range(len(cluster_df))
          fig6 = px.parallel_coordinates(cluster_df,color='sample',color_continuous_scale="picnic",dimensions=cluster_df.columns[:-1],)
          fig6.update_layout(
              plot_bgcolor='black',  # Set the background color to black
              paper_bgcolor='black',  # Set the plot area background color to black
              font_color='white',  # Set the font color to white for better visibility
              title='unnormalized features of cluster: '+str(key) )
          fig6.show()

          # fig7 = go.Figure(go.Parcoords(
          #     line=dict(color=cluster_data[:, 0], showscale=True),
          #     dimensions=[dict(range=[min(cluster_data[:, i]), max(cluster_data[:, i])], label=l, values=cluster_data[:, i]) for i,l in zip( range(len(angular_labels)),angular_labels)],
          # ))
          # fig7.update_layout(title='unnormalized stats features', coloraxis_colorbar=dict(title='Color'))
          # fig7.show()

          ax4.set_xlabel('Demand value')
          ax4.set_ylabel('Density/Frequency')
          ax4.set_title('station''s Kernel Density Plots of cluster: '+str(key))
          ax4.legend()
          fig4.show()

          print('Number of stations in cluster '+ str(key)+' : ', len(cluster_to_stations_dict[key]))
          print('Stations in cluster: ',cluster_to_stations_dict[key])
    ################## par coords plot ##################


    ################## polar plot ##################

    elif(plot_type == 'polar'):
        angular_labels = [ str(l) for l in range( X_train.shape[1] )]
        for key in cluster_to_stations_dict.keys():
          fig0, ax = plt.subplots(figsize=(10,5))
          shapefile_data.plot(ax=ax,edgecolor='black', linewidth=0.9, color='lightblue',figsize=(5,5))

          #fig = go.Figure()
          fig1 = go.Figure()

          d = {}

          feat_min_per_cluster = [1000]*num_features
          feat_max_per_cluster = [-1000]*num_features

          for station in cluster_to_stations_dict[key]:
            ax.scatter(zone_to_gps_dict[int(station)][0], zone_to_gps_dict[int(station)][1], color='red', marker='.')

            y_arr = np.array(df[station].values).reshape(-1)
            # fig = fig.add_trace(go.Scatter(x=np.arange(len(y_arr)), y=y_arr,mode='lines+markers',name=station))
            # fig.update_layout(title='cluster: '+ str(key),xaxis_title="time (Monday 0:00 - Sunday 23:00)",
            # yaxis_title=TS_type,autosize=False,width=1800,height=400, )

            d['r'] = X_train[station_to_index_dict[station]]
            d['theta'] = angular_labels
            fig1.add_trace(go.Scatterpolar(
                r=d['r'],
                theta=d['theta'],
                fill='toself',name=station,
            ))

            sample = X_train[station_to_index_dict[station]]  # Get the current sample
            for f in range(num_features):
                feat_min_per_cluster[f] = min(feat_min_per_cluster[f],sample[f])
                feat_max_per_cluster[f] = max(feat_max_per_cluster[f],sample[f])

          all_cluster_feat_mins = np.append(all_cluster_feat_mins,np.array(feat_min_per_cluster))
          all_cluster_feat_maxs = np.append(all_cluster_feat_maxs,np.array(feat_max_per_cluster))

          fig1.update_layout(title='cluster: '+ str(key),
              polar=dict(
                  radialaxis=dict(visible=True,))#range=[-2, 7]))
              ,height=1000,width=1000,
              showlegend=True
          )

          ax.set_xlabel('Longitude')
          ax.set_ylabel('Latitude')
          ax.set_title('Singapore Stations of cluster '+ str(key))
          fig0.show()
          #fig.show()
          fig1.show()

          print('Number of stations in cluster '+ str(key)+' : ', len(cluster_to_stations_dict[key]))
          print('Stations in cluster: ',cluster_to_stations_dict[key])
    ################## polar plot ##################
    else:
        raise ValueError('only par_coords or are polar allowed')

    all_cluster_feat_mins = np.hstack(all_cluster_feat_mins)
    all_cluster_feat_maxs = np.hstack(all_cluster_feat_maxs)
    print(all_cluster_feat_mins)
    return cluster_to_stations_dict,all_cluster_feat_mins,all_cluster_feat_maxs

#plot_stations_of_clusters(km.labels_, Dim_Red_unmet_60min_outflow_week_df,'outflow')

def plot_silhouette_scores(X_train,labels,metric,linkage,clustering_type):
    unique_clusters = np.unique(labels)
    num_clusters = len(unique_clusters)
    sample_silhouette_values = silhouette_samples(X=X_train, labels=labels,metric=metric)
    silhouette_avg = silhouette_score(X=X_train, labels=labels,metric=metric)
    fig, ax1 = plt.subplots()
    y_lower = 10
    mean_num_zones_dev_in_clusters = []

    mean_num_zones_in_clusters = X_train.shape[0] /num_clusters

    for i in range(num_clusters): # exclude the
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[labels == i]
        ith_cluster_silhouette_values.sort()
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        color = cm.nipy_spectral(float(i) / num_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )
        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples
        mean_num_zones_dev_in_clusters.append(abs(mean_num_zones_in_clusters - np.sum(labels == i) ))

    ax1.set_title("metric: "+metric+", linkage: "+linkage+", silhouette_avg: "+str(silhouette_avg)+", num_clusters: "+str(num_clusters)+", Cluster_type : "+clustering_type )
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")
    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")
    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    num_neg_silhouette_scores = np.sum(sample_silhouette_values < 0 )
    min_silhouette_score = min(sample_silhouette_values)
    dev_from_mean_num_zone = np.sum(np.array(mean_num_zones_dev_in_clusters))
    return silhouette_avg, num_neg_silhouette_scores, min_silhouette_score,dev_from_mean_num_zone


def plot_cluster_representatives(X_train,all_cluster_feat_mins,all_cluster_feat_maxs,plot_type,angular_labels):
    num_features = X_train.shape[1]
    low_list =  all_cluster_feat_mins[num_features:] ## remove outlier cluster
    high_list = all_cluster_feat_maxs[num_features:] ## remove outlier cluster
    num_clusters = int(len(all_cluster_feat_mins[num_features:] )/num_features) ## remove outlier cluster
    color = cm.rainbow(np.linspace(0, 1, num_clusters))

    low = {}
    high = {}
    c = 0
    idx=0
    low[0] = []
    high[0] = []
    for l,h in zip(low_list,high_list):
        low[c].append(l)
        high[c].append(h)
        idx+=1
        if idx%num_features==0:
            c+=1
            low[c] = []
            high[c] = []


    if(plot_type == 'par_coords'): # parallel coordinates
        plt.close('all')

        #fig,ax = plt.subplots(figsize=(20,7))

        cluster_rep_arr = np.empty((num_clusters,num_features))
        adj=0
        for c in range(num_clusters):
            for f in range(num_features):
              cluster_rep_arr[c,f] = np.average([low[c][f],high[c][f]])
              #ax.plot([f+adj]*2,[low[c][f],high[c][f]],c=color[c],marker='*',label=str(c))
            adj += 0.05

        # ax.set_xticks(range(num_features))
        # ax.set_xticklabels([i for i in angular_labels])
        # ax.set_title('Representative points from each cluster for '+str(num_clusters) + ' clusters '+ 'excluding outlier cluster')
        # ax.set_xlabel('TS stats features')
        # ax.set_ylabel('Normalized value')
        # ax.grid(True, linestyle='--')
        # ax.legend()
        # fig.show()


        # ####### TREND statistical Features #######
        # cluster_rep_arr[:,4] = 1- cluster_rep_arr[:,4] # invert
        # cluster_rep_arr[:,5] = 1- cluster_rep_arr[:,5] # invert
        # cluster_rep_arr[:,6] = 1- cluster_rep_arr[:,6] # invert
        # angular_labels[4] = '(1 - norm zero counts)'
        # angular_labels[5] = '(1 - norm kurtosis)'
        # angular_labels[6] = '(1 - norm skewness)'
        # ####### TREND statistical Features #######


        df = pd.DataFrame(cluster_rep_arr, columns=angular_labels)
        df['sample'] = range(len(df))
        fig2 = px.parallel_coordinates(df,color='sample',color_continuous_scale="picnic",dimensions=df.columns[:-1],)
        fig2.update_layout(
            plot_bgcolor='black',
            paper_bgcolor='black',
            font_color='white',
            title='Representative points from each cluster. (excluding outlier cluster)'
        )
        fig2.show()


    elif(plot_type == 'polar'): # polar plot
        plt.close('all')
        angular_labels = [ str(l) for l in range( X_train.shape[1] )]

        cmap = cm.get_cmap('tab20c')
        colors = [mcolors.rgb2hex(cmap(i)[:3]) for i in range(num_clusters)]

        d = {}
        fig1 = go.Figure()

        for c in range(num_clusters):
            color = colors[c % num_clusters]
            d['r'] = low[c]
            d['theta'] = angular_labels
            fig1.add_trace(go.Scatterpolar(
                r=d['r'],
                theta=d['theta'],
                fill='toself',name=str(c)+'_low',
                line=dict(color=color),
            ))
            d['r'] = high[c]
            d['theta'] = angular_labels
            fig1.add_trace(go.Scatterpolar(
                r=d['r'],
                theta=d['theta'],
                fill='toself',name=str(c)+'_high',
                line=dict(color=color),
            ))

        fig1.update_layout(title='Representative DWT points from each cluster. (excluding outlier cluster)',
            polar=dict(
                radialaxis=dict(visible=True,))#range=[-2, 7]))
            ,height=1000,width=1000,
            showlegend=True,
        )
        fig1.show()
    return

def rank_clusterings(all_cluster_feat_mins,all_cluster_feat_maxs,num_clusters,num_features):

  all_cluster_feat_mins = all_cluster_feat_mins.reshape(num_clusters,num_features)
  all_cluster_feat_maxs = all_cluster_feat_maxs.reshape(num_clusters,num_features)

  mean_arr = np.empty((num_clusters,num_features))
  deviation_arr = np.empty((num_clusters,num_features))

  for c in range(num_clusters):
    for f in range(num_features):
      mean = np.mean((all_cluster_feat_mins[c,f],all_cluster_feat_maxs[c,f])) #ignore outlier cluster
      deviation = (all_cluster_feat_maxs[c,f] - all_cluster_feat_mins[c,f])/2
      mean_arr[c,f] = mean
      deviation_arr[c,f] = deviation

  deviation_arr = 1 - deviation_arr
  weights_arr = (deviation_arr.T / deviation_arr.sum(axis=1)).T
  mean_arr = mean_arr*weights_arr

  cluster_magnitudes = np.linalg.norm(mean_arr,2,axis=1)
  ranked_clusters = np.argsort(cluster_magnitudes)
  ranked_clusters = { i:r for i,r in enumerate(ranked_clusters)}
  return ranked_clusters



In [ ]:

def compute_mahalanobis_dist_mat(X):
    # Calculate the covariance matrix
    cov_matrix = np.cov(X.T)
    reg_param = 0.01  # Regularization parameter
    # Calculate the inverse of the covariance matrix
    inv_cov_matrix = np.linalg.inv(cov_matrix + reg_param * np.eye(cov_matrix.shape[0]))
    #inv_cov_matrix = np.linalg.inv(cov_matrix)
    # Calculate the Mahalanobis distance matrix
    mahalanobis_dist_mat = distance.cdist(X, X, metric='mahalanobis', VI=inv_cov_matrix)
    return mahalanobis_dist_mat

In [ ]:
class plot_all():
    def __init__(self,print_msg,X_train):
        self.print_msg = print_msg
        self.X_train = X_train
        return

    def plot_tSNE_2d(self,X_train,print_msg,ax,col):
        ## t-SNE
        plot_kwds = {'alpha' : 0.25, 's' : 10, 'linewidths':0}
        projection = TSNE(n_components=2).fit_transform(X_train)
        ax[col].scatter(*projection.T, **plot_kwds)
        ax[col].set_title('t-SNE, '+ print_msg)
        #plt.show()
        return ax

    def plot_UMAP_2d(self,X_train,print_msg):
        umap_2d = UMAP(n_components=2, init='random', random_state=0)
        proj_2d = umap_2d.fit_transform(X_train)
        fig = go.Scatter(
            x=proj_2d[:, 0],  # X-axis values
            y=proj_2d[:, 1],  # Y-axis values
            mode='markers',
            marker=dict(
                size=5,
                color=np.arange(len(X_train)),  # Color points based on a range of values (e.g., sample index)
                colorscale='Viridis',  # Choose a colorscale
                showscale=True  # Show the colorbar
            ), name=print_msg
        )
        #fig.update_layout(title=print, scene=dict(xaxis_title='Dimension 1', yaxis_title='Dimension 2', zaxis_title='Dimension 3'))
        return fig


    def plot_UMAP_3d(self,X_train,n_neighbors,min_dist,metric):
        umap_3d = UMAP(n_components=3,n_neighbors=n_neighbors,min_dist=min_dist,metric=metric, init='random', random_state=0)
        proj_3d = umap_3d.fit_transform(X_train)
        fig = go.Scatter3d(
            x=proj_3d[:, 0],  # X-axis values from the low-dimensional representation
            y=proj_3d[:, 1],  # Y-axis values from the low-dimensional representation
            z=proj_3d[:, 2],  # Z-axis values from the low-dimensional representation
            mode='markers',
            marker=dict(
                size=5,
                color=np.arange(len(X_train)),  # Color points based on a range of values (e.g., sample index)
                colorscale='Viridis',  # Choose a colorscale
                showscale=True  # Show the colorbar
            ),name=self.print_msg
        )
        #fig.show()
        return fig

    def plot_UMAP_interactive(self,n_neighbors,min_dist,metric):
        umap_3d = UMAP(n_components=3,n_neighbors=n_neighbors,min_dist=min_dist,metric=metric, init='random', random_state=0)
        proj_3d = umap_3d.fit_transform(self.X_train )
        x=proj_3d[:,0]
        y=proj_3d[:,1]
        z=proj_3d[:,2]
        # Create the figure and subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

        # Plot 1 - 2D Plot
        ax1.scatter(x, y,c=np.arange(len(self.X_train )))
        ax1.set_xlabel('X')
        ax1.set_ylabel('Y')
        ax1.set_title(self.print_msg)

        # Plot 2 - 3D Plot
        ax2 = fig.add_subplot(122, projection='3d')
        ax2.scatter(x, y, z, c=np.arange(len(self.X_train )))
        ax2.set_xlabel('X')
        ax2.set_ylabel('Y')
        ax2.set_zlabel('Z')
        ax2.set_title(self.print_msg)

        # Adjust the spacing between subplots
        plt.subplots_adjust(wspace=0.5)

        # Display the plot
        plt.show()
        return


    def plot_ISOMAP_2d(self,X_train,print_msg,n_neighbors=5):
        ## isomap
        # Create an instance of the Isomap class and specify the desired parameters
        n_components = 2  # Number of dimensions in the low-dimensional representation
        # Number of nearest neighbors used in constructing the neighborhood graph
        isomap = Isomap(n_components=n_components, n_neighbors=n_neighbors)
        # Fit the Isomap model to your data
        isomap_embedding = isomap.fit_transform(X_train)
        # Access the low-dimensional representation
        low_dim_data = isomap_embedding
        fig = go.Scatter(
            x=low_dim_data[:, 0],  # X-axis values
            y=low_dim_data[:, 1],  # Y-axis values
            mode='markers',
            marker=dict(
                size=5,
                color=np.arange(len(X_train)),  # Color points based on a range of values (e.g., sample index)
                colorscale='Viridis',  # Choose a colorscale
                showscale=True  # Show the colorbar
            ), name=print_msg
        )
        #fig.update_layout(title=print, scene=dict(xaxis_title='Dimension 1', yaxis_title='Dimension 2', zaxis_title='Dimension 3'))
        #fig.show()
        return fig

    def plot_ISOMAP_3d(self,X_train,print_msg,n_neighbors=5):
        ## isomap
        # Create an instance of the Isomap class and specify the desired parameters
        n_components = 3  # Number of dimensions in the low-dimensional representation
        n_neighbors = 5  # Number of nearest neighbors used in constructing the neighborhood graph
        isomap = Isomap(n_components=n_components, n_neighbors=n_neighbors)
        # Fit the Isomap model to your data
        isomap_embedding = isomap.fit_transform(X_train)
        # Access the low-dimensional representation
        low_dim_data = isomap_embedding
        # Create a 3D scatter plot using Plotly
        fig = go.Scatter3d(
            x=low_dim_data[:, 0],  # X-axis values from the low-dimensional representation
            y=low_dim_data[:, 1],  # Y-axis values from the low-dimensional representation
            z=low_dim_data[:, 2],  # Z-axis values from the low-dimensional representation
            mode='markers',
            marker=dict(
                size=5,
                color=np.arange(len(X_train)),  # Color points based on a range of values (e.g., sample index)
                colorscale='Viridis',  # Choose a colorscale
                showscale=True  # Show the colorbar
            ),name=print_msg
        )
        #fig.show()
        return fig

    def plot_hist_n_corr(self,X_train):
        X_train_df = pd.DataFrame(X_train)
        ######### histogram plot #########
        # check to see if any features are all zeros
        print(self.print_msg+' individual feature Histogram')
        X_train_df.hist(figsize=(10,10))
        plt.show()
        # ######### plot correlation #########
        # # perform dimensionality reduction by removing highly correlated features.
        X_train_df_corr = X_train_df.corr()
        X_train_corr = X_train_df_corr.to_numpy()
        fig, ax = plt.subplots(figsize=(15,15))
        cax = ax.matshow(X_train_corr, cmap='coolwarm')
        cbar = fig.colorbar(cax)
        # Loop over the data and add text annotations
        for i in range(X_train_corr.shape[0]):
            for j in range(X_train_corr.shape[1]):
                text = ax.text(j, i, f'{X_train_corr[i, j]:.3f}', ha='center', va='center')
        ax.set_title(self.print_msg+' features correlation matrix')
        plt.show()
        corr_threshold = 0.95 ## Warn multicollinearity
        if ((X_train_corr.any() >= corr_threshold) and (X_train_corr.shape[0] > 1) and (X_train_corr.any() <= 0.9999) ):
            warnings.warn("Warning: some correlation is higher than "+str(corr_threshold))
        h = np.histogram(X_train_df.dropna())
        if  ( ( (  np.count_nonzero(h[0]) ) / h[0].shape[0]) < (1/3)):
            warnings.warn("Warning: highly skewed feature, (provies no information!) ")
        return

In [ ]:
h = np.histogram(np.array([2,5,4,6,4,7,1,2,3]))



In [ ]:
"""                                                                     FEATURE Extraction
Feature extracted: Interpretable Time series statistical features

1) High/Low average demand for each day of 7 days


"""

def select_days_in_dataframe(df,days_type='all_days'):
    if days_type == 'all_days':
        return df
    elif days_type == 'week_days':
        df = df[ (df['weekday'] == 'Monday' ) | (df['weekday'] == 'Tuesday' ) | (df['weekday'] == 'Wednesday' ) | (df['weekday'] == 'Thursday' ) ]
        return df
    elif days_type == 'week_ends':
        df = df[ (df['weekday'] == 'Friday' ) | (df['weekday'] == 'Saturday' ) | (df['weekday'] == 'Sunday' ) ]
        return df
    else:
      raise ValueError('Must be one of the 3: ""all_days"" or ""week_days"" or ""week_ends""')


#######################################################################################################################
""" TREND STATISTICAL FEATURES functions"""
def calculate_mean_crossings(list_values):
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) >= np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return no_mean_crossings

def calculate_max_deviation_from_n95(list_values):
    return np.max(list_values) - np.percentile(list_values,0.95)

def calculate_zero_demands(list_values):
    return np.where(list_values==0)[0].size

def median_of_lists(list_of_lists):
    combined_lists  = np.array([sublist for sublist in list_of_lists])
    medians = np.median(combined_lists,axis=0)
    return medians

def no_mean_crossing_demand(df):
    return df.apply(calculate_mean_crossings,axis=0)

def average_demand(df):
    #use this method instead of sample mean over all sundays. because as sample increases mean goes to zero
    return df.mean(axis=0)

def peak_n95_demand(df):
    return df.quantile(q=0.95,axis=0)

def max_demand(df):
    return df.max(axis=0)

def outlier_demands(df):
    return df.apply(calculate_max_deviation_from_n95,axis=0)

def count_zero_demands(df):
    return df.apply(calculate_zero_demands,axis=0)

def kurt_of_demands(df):
    return df.apply(stats.kurtosis,axis=0)

def skew_of_demands(df):
    return df.skew(axis=0)

def total_demands(df):
    return df.sum(axis=0)

## all are zero ##
# def mode_demands(df):
#     df = df.drop(columns=['week','hour','weekday'])
#     print(df.mode(axis=0))
#     return df.mode(axis=0)

def range_demands(df):
    return df.max(axis=0) - df.min(axis=0)
""" TREND STATISTICAL FEATURES functions"""
#######################################################################################################################



#######################################################################################################################
""" SEASONALITY STATISTICAL FEATURES functions"""

def std_demand(df):
    return df.std(axis=0)

def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy= stats.entropy(probabilities)
    return entropy

def entropy_demands(df):
    return df.apply(calculate_entropy,axis=0)

def calculate_positive_peak_autocorrelation_of_demand(list_values):
    view_lags = len(list_values)
    feat_len = 5
    acf1 = acf(list_values, nlags=view_lags)
    peak_idx_high = np.array((find_peaks(acf1,height=(-0.7,0.7))[0]),dtype=int)
    x = list(range(0,len(acf1)))
    signific_high_idx = np.where(acf1[peak_idx_high] > 1.96/np.sqrt(len(x)))
    positive_autocorr_lags = peak_idx_high[signific_high_idx][:feat_len]
    positive_autocorr_lags = np.concatenate((positive_autocorr_lags,np.zeros(feat_len-len(positive_autocorr_lags))))
    return list(positive_autocorr_lags)

def calculate_negative_peak_autocorrelation_of_demand(list_values):
    view_lags = len(list_values)
    feat_len = 5
    acf1 = acf(list_values, nlags=view_lags)
    peak_idx_low = np.array((find_peaks(-1*acf1,height=(-0.7,0.7))[0]),dtype=int)
    x = list(range(0,len(acf1)))
    signific_low_idx = np.where(acf1[peak_idx_low] < (-1*1.96)/np.sqrt(len(x)))
    negative_autocorr_lags = peak_idx_low[signific_low_idx][:feat_len]
    negative_autocorr_lags = np.concatenate((negative_autocorr_lags,np.zeros(feat_len-len(negative_autocorr_lags))))
    return list(negative_autocorr_lags)

# def calculate_positive_peak_partial_autocorrelation_of_demand(list_values):
#     view_lags = int(len(list_values)/2) - 1
#     feat_len = 10
#     pacf1 = pacf(list_values, nlags=view_lags)
#     peak_idx_high = np.array((find_peaks(pacf1,height=(-0.7,0.7))[0]),dtype=int)
#     x = list(range(0,len(pacf1)))
#     signific_high_idx = np.where(pacf1[peak_idx_high] > 1.96/np.sqrt(len(x)))
#     positive_partial_autocorr_lags = peak_idx_high[signific_high_idx][:feat_len]
#     positive_partial_autocorr_lags = np.concatenate((positive_partial_autocorr_lags,np.zeros(feat_len-len(positive_partial_autocorr_lags))))
#     return list(positive_partial_autocorr_lags)

# def calculate_negative_peak_partial_autocorrelation_of_demand(list_values):
#     view_lags = int(len(list_values)/2) - 1
#     feat_len = 10
#     pacf1 = pacf(list_values, nlags=view_lags)
#     peak_idx_low = np.array((find_peaks(-1*pacf1,height=(-0.7,0.7))[0]),dtype=int)
#     x = list(range(0,len(pacf1)))
#     signific_low_idx = np.where(pacf1[peak_idx_low] < (-1*1.96)/np.sqrt(len(x)))
#     negative_partial_autocorr_lags = peak_idx_low[signific_low_idx][:feat_len]
#     negative_partial_autocorr_lags = np.concatenate((negative_partial_autocorr_lags,np.zeros(feat_len-len(negative_partial_autocorr_lags))))
#     return list(negative_partial_autocorr_lags)

def demands_positive_autocorr_lags(df):
    #df['Date'] = df.index
    daily_pos_autocorr_lags = df.apply(calculate_positive_peak_autocorrelation_of_demand, axis=0)
    return daily_pos_autocorr_lags.values.T

def demands_negative_autocorr_lags(df):
    #df['Date'] = df.index
    daily_neg_autocorr_lags = df.apply(calculate_negative_peak_autocorrelation_of_demand, axis=0)
    return daily_neg_autocorr_lags.values.T

# def demands_positive_partial_autocorr_lags(df):
#     #df['Date'] = df.index
#     daily_pos_partial_autocorr_lags = df.apply(calculate_positive_peak_partial_autocorrelation_of_demand, axis=0)
#     return daily_pos_partial_autocorr_lags.values.T

# def demands_negative_partial_autocorr_lags(df):
#     #df['Date'] = df.index
#     daily_neg_partial_autocorr_lags = df.apply(calculate_negative_peak_partial_autocorrelation_of_demand, axis=0)
#     return daily_neg_partial_autocorr_lags.values.T

def fft_of_demand(df):
    stations = df.columns
    num_common_freq=180
    top_x_frequencies = 11
    freq_rounding_num = 6
    common_freq_set = set()
    X_train = np.empty(shape=(0,top_x_frequencies-1)) # removed dc

    for station in stations:
        # Perform the FFT on the data
        fft_result = np.fft.fft(df[station].values)
        # Compute the magnitudes of the FFT result
        fft_magnitudes = np.abs(fft_result)
        # Generate the corresponding frequency axis
        sampling_rate = 1.0  # Assuming a sampling rate of 1.0 (change as needed)
        frequency_axis = np.fft.fftfreq(len(fft_result), 1 / sampling_rate)
        positive_freq_indices = np.where(frequency_axis >= 0)

        """ TOP periods/frequencies in the time series"""
        # Sort the magnitudes and frequencies in descending order
        sorted_indices = np.argsort(fft_magnitudes[positive_freq_indices])[::-1]
        sorted_magnitudes = fft_magnitudes[positive_freq_indices][sorted_indices]
        sorted_frequencies = frequency_axis[positive_freq_indices][sorted_indices]
        # Select the top periods

        top_freq = sorted_frequencies[2:top_x_frequencies+1] # REMOVE dc/mean component
        top_periods = 1/top_freq
        top_periods = [round(p) for p in top_periods]
        #print(f'Top FFT periods in demand are : {top_periods}')
        X_train = np.vstack((X_train,top_periods))
    return X_train
""" SEASONALITY STATISTICAL FEATURES functions"""
#######################################################################################################################



In [ ]:
"""                                                                     FEATURE Extraction
Feature extracted: Interpretable Time series statistical features

1) High/Low average demand for each day of 7 days


"""

################## Select one of ""all_days"" or ""week_days" (Mon,Tue,Wed,Thur) or "week_ends" (Fri,Sat,Sun) to generate trend/seasonality statistical features ##################
""" Select DAYS of the Dataframe to use"""
zone_met_10m_outflow_df = select_days_in_dataframe(manhat_demand,days_type='all_days')
################## Select one of ""all_days"" or ""week_days" (Mon,Tue,Wed,Thur) or "week_ends" (Fri,Sat,Sun) to generate trend/seasonality statistical features ##################



######################################################################################################################
""" TREND STATISTICAL FEATURES calculation"""

avg_dem = average_demand(zone_met_10m_outflow_df)
avg_dem_ts_stats_feat_transform = feat_transformation()
X_train_normalized_avg_dem = avg_dem_ts_stats_feat_transform.feature_normalization(  np.array([avg_dem.to_numpy()]).T  )
print_msg = 'avg_dem'
plot_all(print_msg,X_train_normalized_avg_dem).plot_hist_n_corr(X_train_normalized_avg_dem)
X_train_inv_normalized_avg_dem = avg_dem_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_avg_dem)
# print_msg = 'every_days_avg_dem_non_normalized'
# plot_all(print_msg,X_train_inv_normalized_avg_dem).plot_hist_n_corr(X_train_inv_normalized_avg_dem)

no_mean_cross_dem = no_mean_crossing_demand(zone_met_10m_outflow_df)
num_mean_cross_ts_stats_feat_transform = feat_transformation()
X_train_normalized_mean_cross = num_mean_cross_ts_stats_feat_transform.feature_normalization(  np.array([no_mean_cross_dem.to_numpy()]).T  )
print_msg = 'no_mean_crossing_dem'
plot_all(print_msg,X_train_normalized_mean_cross).plot_hist_n_corr(X_train_normalized_mean_cross)
X_train_inv_normalized_mean_cross = num_mean_cross_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_mean_cross)

n95_dem = peak_n95_demand(zone_met_10m_outflow_df)
n95_ts_stats_feat_transform = feat_transformation()
X_train_normalized_n95 = n95_ts_stats_feat_transform.feature_normalization(  np.array([n95_dem.to_numpy()]).T   )
print_msg = 'n95_dem'
plot_all(print_msg,X_train_normalized_n95).plot_hist_n_corr(X_train_normalized_n95)
X_train_inv_normalized_n95 = n95_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_n95)

max_dem = max_demand(zone_met_10m_outflow_df)
max_dem_ts_stats_feat_transform = feat_transformation()
X_train_normalized_max = max_dem_ts_stats_feat_transform.feature_normalization(    np.array([max_dem.to_numpy()]).T  )
print_msg = 'max_dem'
plot_all(print_msg,X_train_normalized_max).plot_hist_n_corr(X_train_normalized_max)
X_train_inv_normalized_max = max_dem_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_max)

max_deviation_from_n95 = outlier_demands(zone_met_10m_outflow_df)
max_dev_n95_ts_stats_feat_transform = feat_transformation()
X_train_normalized_max_dev_n95 = max_dev_n95_ts_stats_feat_transform.feature_normalization(  np.array([max_deviation_from_n95.to_numpy()]).T   )
print_msg = 'max_dev_from_n95_dem'
plot_all(print_msg,X_train_normalized_max_dev_n95).plot_hist_n_corr(X_train_normalized_max_dev_n95)
X_train_inv_normalized_max_dev_n95 = max_dev_n95_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_max_dev_n95)

zero_dem = count_zero_demands(zone_met_10m_outflow_df)
zero_dem_ts_stats_feat_transform = feat_transformation()
X_train_normalized_zeros = zero_dem_ts_stats_feat_transform.feature_normalization(   np.array([zero_dem.to_numpy()]).T   )
print_msg = 'zero_dem'
plot_all(print_msg,X_train_normalized_zeros).plot_hist_n_corr(X_train_normalized_zeros)
X_train_inv_normalized_zeros = zero_dem_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_zeros)


kurt = kurt_of_demands(zone_met_10m_outflow_df)
kurt_ts_stats_feat_transform = feat_transformation()
X_train_normalized_kurt = kurt_ts_stats_feat_transform.feature_normalization(   np.array([kurt.to_numpy()]).T  )
print_msg = 'kurt_dem'
plot_all(print_msg,X_train_normalized_kurt).plot_hist_n_corr(X_train_normalized_kurt)
X_train_inv_normalized_kurt = kurt_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_kurt)

skew = skew_of_demands(zone_met_10m_outflow_df)
skew_ts_stats_feat_transform = feat_transformation()
X_train_normalized_skew = skew_ts_stats_feat_transform.feature_normalization(   np.array([skew.to_numpy()]).T  )
print_msg = 'skew_dem'
plot_all(print_msg,X_train_normalized_skew).plot_hist_n_corr(X_train_normalized_skew)
X_train_inv_normalized_skew = skew_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_skew)


tot_dem = total_demands(zone_met_10m_outflow_df)
tot_dem_ts_stats_feat_transform = feat_transformation()
X_train_normalized_tot_dem = tot_dem_ts_stats_feat_transform.feature_normalization(   np.array([tot_dem.to_numpy()]).T  )
print_msg = 'tot_dem'
plot_all(print_msg,X_train_normalized_tot_dem).plot_hist_n_corr(X_train_normalized_tot_dem)
X_train_inv_normalized_tot_dem = tot_dem_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_tot_dem)

range_dem = range_demands(zone_met_10m_outflow_df)
range_dem_ts_stats_feat_transform = feat_transformation()
X_train_normalized_range_dem = range_dem_ts_stats_feat_transform.feature_normalization(   np.array([range_dem.to_numpy()]).T  )
print_msg = 'range_dem'
plot_all(print_msg,X_train_normalized_range_dem).plot_hist_n_corr(X_train_normalized_range_dem)
X_train_inv_normalized_range_dem = range_dem_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_range_dem)


# X_train_normalized_all_trend_statistic_features = np.concatenate((X_train_normalized_avg_dem,X_train_normalized_mean_cross,X_train_normalized_n95,X_train_normalized_max,\
#                                                                   X_train_normalized_max_dev_n95,X_train_normalized_zeros,X_train_normalized_kurt,\
#                                                                   X_train_normalized_tot_dem,X_train_normalized_range_dem,X_train_normalized_skew),axis=1)
#### After removing correlated features
X_train_normalized_all_trend_statistic_features = np.concatenate((X_train_normalized_avg_dem,X_train_normalized_mean_cross,X_train_normalized_n95,\
                                                                  X_train_normalized_zeros,X_train_normalized_skew),axis=1)

X_train_inverse_normalized_all_trend_statistic_features = np.concatenate((X_train_inv_normalized_avg_dem,X_train_inv_normalized_mean_cross,X_train_inv_normalized_n95,\
                                                                  X_train_inv_normalized_zeros,X_train_inv_normalized_skew),axis=1)

print_msg = 'all_trend_statistics features'
plot_all(print_msg,X_train_normalized_all_trend_statistic_features).plot_hist_n_corr(X_train_normalized_all_trend_statistic_features)


""" TREND STATISTICAL FEATURES calculation"""
######################################################################################################################







# #######################################################################################################################
# """ SEASONALITY STATISTICAL FEATURES calculation"""

# avg_dem = average_demand(select_days_station_met_60m_outflow_df)
# avg_dem_ts_stats_feat_transform = feat_transformation()
# X_train_normalized_avg_dem = avg_dem_ts_stats_feat_transform.feature_normalization(  np.array([avg_dem.to_numpy()]).T  )
# print_msg = 'avg_dem'
# #plot_all(print_msg,X_train_normalized_avg_dem).plot_hist_n_corr(X_train_normalized_avg_dem)
# X_train_inv_normalized_avg_dem = avg_dem_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_avg_dem)
# # print_msg = 'every_days_avg_dem_non_normalized'
# # plot_all(print_msg,X_train_inv_normalized_avg_dem).plot_hist_n_corr(X_train_inv_normalized_avg_dem)

# no_mean_cross_dem = no_mean_crossing_demand(station_met_outflow_60min_df)
# num_mean_cross_ts_stats_feat_transform = feat_transformation()
# X_train_normalized_mean_cross = num_mean_cross_ts_stats_feat_transform.feature_normalization(  np.array([no_mean_cross_dem.to_numpy()]).T  )
# print_msg = 'no_mean_crossing_dem'
# #plot_all(print_msg,X_train_normalized_mean_cross).plot_hist_n_corr(X_train_normalized_mean_cross)
# X_train_inv_normalized_mean_cross = num_mean_cross_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_mean_cross)

# std_dem = std_demand(station_met_outflow_60min_df)
# std_dem_ts_stats_feat_transform = feat_transformation()
# X_train_normalized_std = std_dem_ts_stats_feat_transform.feature_normalization(   np.array([std_dem.to_numpy()]).T   )
# print_msg = 'std_dem'
# #plot_all(print_msg,X_train_normalized_std).plot_hist_n_corr(X_train_normalized_std)
# X_train_inv_normalized_std = std_dem_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_std)

# entropy_dem = entropy_demands(station_met_outflow_60min_df)
# entropy_dem_ts_stats_feat_transform = feat_transformation()
# X_train_normalized_entropy_dem = entropy_dem_ts_stats_feat_transform.feature_normalization(   np.array([entropy_dem.to_numpy()]).T  )
# print_msg = 'entropy_dem'
# #plot_all(print_msg,X_train_normalized_entropy_dem).plot_hist_n_corr(X_train_normalized_entropy_dem)
# X_train_inv_normalized_entropy_dem = entropy_dem_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_entropy_dem)

# pos_autocorr_lags = demands_positive_autocorr_lags(station_met_outflow_60min_df)
# pos_autocorr_lags_ts_stats_feat_transform = feat_transformation()
# X_train_normalized_pos_autocorr_lags = pos_autocorr_lags_ts_stats_feat_transform.feature_normalization(pos_autocorr_lags)
# print_msg = 'positive_autocorr_lags'
# #plot_all(print_msg,X_train_normalized_pos_autocorr_lags).plot_hist_n_corr(X_train_normalized_pos_autocorr_lags)
# X_train_inv_normalized_pos_autocorr_lags = pos_autocorr_lags_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_pos_autocorr_lags)

# neg_autocorr_lags = demands_negative_autocorr_lags(station_met_outflow_60min_df)
# neg_autocorr_lags_ts_stats_feat_transform = feat_transformation()
# X_train_normalized_neg_autocorr_lags = neg_autocorr_lags_ts_stats_feat_transform.feature_normalization(neg_autocorr_lags)
# print_msg = 'negative_autocorr_lags'
# #plot_all(print_msg,X_train_normalized_neg_autocorr_lags).plot_hist_n_corr(X_train_normalized_neg_autocorr_lags)
# X_train_inv_normalized_neg_autocorr_lags = neg_autocorr_lags_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_neg_autocorr_lags)


# peak_fft_periods = fft_of_demand(station_met_outflow_60min_df)
# fft_ts_stats_feat_transform = feat_transformation()
# X_train_normalized_fft = fft_ts_stats_feat_transform.feature_normalization(peak_fft_periods)
# print_msg = 'fft_dem'
# #plot_all(print_msg,X_train_normalized_fft).plot_hist_n_corr(X_train_normalized_fft)
# X_train_inv_normalized_fft = fft_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_fft)

# # X_train_inv_normalized_fft = fft_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized_fft)
# # print_msg = 'fft_non_normalized'
# # plot_all(print_msg).plot_hist_n_corr(X_train_inv_normalized_fft)

# # ####### Takes too long to Compute ########
# # pos_partial_autocorr_lags = demands_positive_partial_autocorr_lags(station_met_outflow_60min_df)
# # pos_partial_autocorr_lags_ts_stats_feat_transform = feat_transformation()
# # X_train_normalized_pos_partial_autocorr_lags = pos_partial_autocorr_lags_ts_stats_feat_transform.feature_normalization(pos_partial_autocorr_lags)
# # print_msg = 'positive_partial_autocorr_lags'
# # plot_all(print_msg,X_train_normalized_neg_partial_autocorr_lags).plot_hist_n_corr(X_train_normalized_pos_partial_autocorr_lags)

# # ####### Takes too long to Compute ########
# # neg_partial_autocorr_lags = demands_negative_partial_autocorr_lags(station_met_outflow_60min_df)
# # neg_partial_autocorr_lags_ts_stats_feat_transform = feat_transformation()
# # X_train_normalized_neg_partial_autocorr_lags = neg_partial_autocorr_lags_ts_stats_feat_transform.feature_normalization(neg_partial_autocorr_lags)
# # print_msg = 'negative_partial_autocorr_lags'
# # plot_all(print_msg,X_train_normalized_neg_partial_autocorr_lags).plot_hist_n_corr(X_train_normalized_neg_partial_autocorr_lags)


# # X_train_normalized_all_seasonality_statistic_features = np.concatenate((X_train_normalized_avg_dem,X_train_normalized_mean_cross,X_train_normalized_std,X_train_normalized_entropy_dem,\
# #                                                                   X_train_normalized_pos_autocorr_lags,X_train_normalized_neg_autocorr_lags,X_train_normalized_fft),axis=1)

# # # After removing correlated features
# # X_train_normalized_all_seasonality_statistic_features = np.concatenate((X_train_normalized_avg_dem,X_train_normalized_mean_cross,X_train_normalized_pos_autocorr_lags,\
# #                                                                         X_train_normalized_fft),axis=1)
# # X_train_inverse_normalized_all_seasonality_statistic_features = np.concatenate((X_train_inv_normalized_avg_dem,X_train_inv_normalized_mean_cross,\
# #                                                                                 X_train_inv_normalized_pos_autocorr_lags,X_train_inv_normalized_fft),axis=1)

# X_train_normalized_all_seasonality_statistic_features = X_train_normalized_pos_autocorr_lags ## np.concatenate((X_train_normalized_fft,X_train_normalized_mean_cross,),axis=1)
# X_train_inverse_normalized_all_seasonality_statistic_features = X_train_inv_normalized_pos_autocorr_lags ##np.concatenate((X_train_inv_normalized_avg_dem,X_train_inv_normalized_mean_cross,),axis=1)

# print_msg = 'all_seasonality_statistics features'
# plot_all(print_msg,X_train_normalized_all_seasonality_statistic_features).plot_hist_n_corr(X_train_normalized_all_seasonality_statistic_features)

# """ SEASONALITY STATISTICAL FEATURES calculation"""
# ######################################################################################################################


In [ ]:
"""                                                                     PLOT UMAP for everyday_avg_demand Time series statistical features

"""

plt.close('all')
X_train_normalized = X_train_normalized_all_trend_statistic_features
X_train_inv_normalized =X_train_inverse_normalized_all_trend_statistic_features
# X_train_normalized = X_train_normalized_all_seasonality_statistic_features
# X_train_inv_normalized =X_train_inverse_normalized_all_seasonality_statistic_features

# print_msg = ''
# plotter_avg_dem = plot_all(print_msg,X_train_normalized)

# interactive_UMAP_plot=interactive(plotter_avg_dem.plot_UMAP_interactive,X_train=X_train_normalized,print_msg=print_msg,n_neighbors=(10,100,20),min_dist=(0,1.0,0.1),metric=['euclidean','mahalanobis','manhattan'])
# display(interactive_UMAP_plot)



In [ ]:
"""                                                                    ### DETAILED 3d UMAP for everyday_avg_demand Time series statistical features

use above to get correct n_neighbors ,min_dist and metric to visuzalize in detain
"""
# plt.close('all')

# fig_umap_3d = sp.make_subplots(specs=[[{'type': 'surface'} for i in range(1)]])

# fig_umap_3d.add_trace(plotter_avg_dem.plot_UMAP_3d(X_train_normalized,10,0.1,'euclidean'))

# fig_umap_3d.update_layout(height=900 , width=900,title='UMAP 3d,'+print_msg)
# fig_umap_3d.show()

In [ ]:
"""
Parameter search for best clustering using TS statistical Features

"""

best_score = 0
best_score_num_cluster_dict = {}
cov_mat = np.cov(X_train_normalized.T)

for metric in ['euclidean','manhattan','mahalanobis']:# need to implement mahalanobis distance
    if metric != 'mahalanobis':
        for min_cluster_size in range(5,80,5):
            for min_samples in range(1,40,2):
                for cluster_selection_method in ['eom','leaf']:
                    # for each combination of parameters of hdbscan
                    hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,min_samples=min_samples,
                                          cluster_selection_method=cluster_selection_method, metric=metric,
                                          gen_min_span_tree=True).fit(X_train_normalized)
                    # DBCV score
                    score = hdb.relative_validity_
                    outliers = (np.array(hdb.labels_)==-1).sum()
                    best_score_num_cluster_dict[score] = [len(np.unique(hdb.labels_)),min_cluster_size,min_samples,cluster_selection_method,metric,outliers]
                    #print(f'score: {score}, num of unique clusters: {len(np.unique(hdb.labels_))}' )
                    # if we got a better DBCV, store it and the parameters
                    if score > best_score:
                        best_score = score
                        best_parameters = {'min_cluster_size': min_cluster_size,
                                  ' min_samples':  min_samples, 'cluster_selection_method': cluster_selection_method,
                                  'metric': metric}
    else: ## mahalanobis distance
        for min_cluster_size in range(5,80,5):
            for min_samples in range(1,40,2):
                for cluster_selection_method in ['eom','leaf']:
                    # for each combination of parameters of hdbscan
                    hdb = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size,min_samples=min_samples,
                                          cluster_selection_method=cluster_selection_method, metric=metric,V=cov_mat,
                                          gen_min_span_tree=True).fit(X_train_normalized)
                    # DBCV score
                    score = hdb.relative_validity_
                    outliers = (np.array(hdb.labels_)==-1).sum()
                    best_score_num_cluster_dict[score] = [len(np.unique(hdb.labels_)),min_cluster_size,min_samples,cluster_selection_method,metric,outliers]
                    #print(f'score: {score}, num of unique clusters: {len(np.unique(hdb.labels_))}' )
                    # if we got a better DBCV, store it and the parameters
                    if score > best_score:
                        best_score = score
                        best_parameters = {'min_cluster_size': min_cluster_size,
                                  ' min_samples':  min_samples, 'cluster_selection_method': cluster_selection_method,
                                  'metric': metric}


print("Best DBCV score: {:.3f}".format(best_score))
print("Best parameters: {}".format(best_parameters))

In [ ]:
sorted(best_score_num_cluster_dict.items())[-40:]

In [ ]:
"""
Further eliminate some Parameter combination for every_days_avg_dem TS statistical Features based on best DBCV score

"""

hdbscan_best_score_comb = []
dbcv_threshold = 0.05
min_num_clusters_threshold = 3
max_num_clusters_threshold = 7
num_outliers_margin = X_train_normalized.shape[0]/2.6 #tolerable_num_outliers

for comb in sorted(best_score_num_cluster_dict.items())[-40:]:
    dbcv = comb[0]
    num_outliers = comb[1][5]
    num_clusters = comb[1][0]
    if ((num_outliers < num_outliers_margin) and (num_clusters < max_num_clusters_threshold) and (num_clusters > min_num_clusters_threshold) and (dbcv > dbcv_threshold) ):
        hdbscan_best_score_comb.append(comb)
hdbscan_best_score_comb



In [ ]:
"""
Plot Minimum spanning tree and dbscan cluster hierarchy from every_days_avg_dem  TS statistical features

SHORTLISTED hdbscan clustering parameters combination for stats features

"""


plt.close('all')
clustering_type = 'HDBSCAN - euclidean metric'

if (hdbscan_best_score_comb[-1][1][4] == 'mahalanobis'):
    hdbscan_model = hdbscan.HDBSCAN(min_cluster_size= hdbscan_best_score_comb[-1][1][1], min_samples= hdbscan_best_score_comb[-1][1][2],cluster_selection_method=hdbscan_best_score_comb[-1][1][3], metric=hdbscan_best_score_comb[-1][1][4], gen_min_span_tree=True,V=cov_mat,)
else:
    hdbscan_model = hdbscan.HDBSCAN(min_cluster_size= hdbscan_best_score_comb[-1][1][1], min_samples= hdbscan_best_score_comb[-1][1][2],cluster_selection_method=hdbscan_best_score_comb[-1][1][3], metric=hdbscan_best_score_comb[-1][1][4], gen_min_span_tree=True)#,V=cov_mat,)


hdbscan_model.fit(X_train_normalized)

fig = plt.figure(figsize=(10,5))
# Plotting the minimum spanning tree in the first subplot
hdbscan_model.minimum_spanning_tree_.plot(
    edge_cmap='viridis',
    edge_alpha=0.6,
    edge_linewidth=2
)
fig.show()

fig = plt.figure(figsize=(20,10))
# Plotting the condensed tree in the second subplot
hdbscan_model.condensed_tree_.plot(
    select_clusters=True,
    selection_palette=sns.color_palette(),
)
fig.show()

print(hdbscan_model.labels_)

plot_silhouette_scores(X_train_normalized,hdbscan_model.labels_,'euclidean','non',clustering_type)

In [ ]:
"""
Visualize every_days_avg_dem Time series statistical features of all clusters using parcoords plot

"""

plt.close('all')
plot_title = 'TS statistical features'
mother = 'rbio2.2'
limit_level = 2
trend_angular_labels = ["Average Demand",'no_mean_crossing', "n95 demand", "Zero demands", "skewness"]
# seasonality_angular_labels = [
# 'pos_ac_lag0',
#  'pos_ac_lag1',
#  'pos_ac_lag2',
#  'pos_ac_lag3',
#  'pos_ac_lag4',]
#seasonality_angular_labels = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9']


#X_train = avg_dem_ts_stats_feat_transform.inv_feature_normalization(X_train_normalized)

#station_met_outflow_60min_df = station_met_outflow_60min_df.drop(columns=['week','weekday','hour'])
clustering_result_dict,all_cluster_feat_mins,all_cluster_feat_maxs = plot_stations_of_clusters(hdbscan_model.labels_, zone_met_10m_outflow_df,'met_demand',X_train_normalized,X_train_inv_normalized,'par_coords',trend_angular_labels,plot_title,mother,limit_level)



In [ ]:
"""
Visualize cluster representatives from every_days_avg_dem Time series statistical features using parcoords plot

"""

plot_cluster_representatives(X_train_normalized, all_cluster_feat_mins, all_cluster_feat_maxs,'par_coords',trend_angular_labels)

In [ ]:
"""
Rank cluster from "highest demand" to "lowest demand" vaguely without the outlier.
"""
num_clusters = len(np.unique(hdbscan_model.labels_))
num_features = X_train_normalized.shape[1]

num_clusters = num_clusters - 1 #remove outlier cluster
all_cluster_feat_mins = all_cluster_feat_mins[num_features:].reshape(num_clusters,num_features)
all_cluster_feat_maxs = all_cluster_feat_maxs[num_features:].reshape(num_clusters,num_features)

ranked_clusters = rank_clusterings(all_cluster_feat_mins,all_cluster_feat_maxs,num_clusters,num_features) # exclude outliers
ranked_clusters

In [ ]:
"""
Display different ranked clusters on the same map
"""

colormap = plt.cm.RdYlGn  # Diverging red to green colormap
norm = mcolors.Normalize(vmin=min(ranked_clusters.keys()), vmax=max(ranked_clusters.keys()))

map_center = [40.7831, -73.9712]  # Manhattan coordinates (latitude, longitude)
map_zoom = 12  # Adjust the zoom level as needed
my_map = folium.Map(location=map_center, zoom_start=map_zoom,width='70%', height='70%')

marker_color = [ 'black','darkred','red','lightred','lightgreen','green','darkgreen',    'lightblue',    'blue',    'darkblue',]

for cluster in clustering_result_dict.keys(): # outliers included
  stations = clustering_result_dict[cluster]
  color = mcolors.to_hex(colormap(norm(cluster)))
  for station in stations:
    folium.Marker(location=zone_to_gps_dict[int(station)], popup=str(station)+': '+color+'-'+str(cluster), icon=folium.Icon(color=marker_color[cluster+1]),tooltip=str(cluster)).add_to(my_map)
display(my_map)

In [ ]:
"""
Identify aggolmeratic clustering parameter combination with best silhouette score for trend and seasonality stat features

"""

plt.close('all')

num_clusters_list = range(4,8,1)
clustering_type = 'Agglomerative'

agglomerative_result_comb = []

methods = ['single','complete','average']#['single','complete','average','weighted',]
metrics = ['euclidean','cityblock']
for metric in metrics:
    for method in methods:
        for num_clusters in num_clusters_list:
            model = AgglomerativeClustering(n_clusters=num_clusters, metric=metric, linkage=method)
            model.fit(X_train_normalized)
            labels = model.labels_
            silhouette_avg, num_neg_silhouette_scores, min_silhouette_score,dev_from_mean_num_zone = plot_silhouette_scores(X_train_normalized,labels,metric,method,clustering_type)
            agglomerative_result_comb.append([silhouette_avg,num_neg_silhouette_scores, min_silhouette_score,metric,method,num_clusters,dev_from_mean_num_zone ])


methods = ['ward']#['median','centroid','ward']
metrics = ['euclidean']
for method in methods:
    for metric in metrics:
        for num_clusters in num_clusters_list:
            model = AgglomerativeClustering(n_clusters=num_clusters, metric=metric, linkage=method)
            model.fit(X_train_normalized)
            labels = model.labels_
            silhouette_avg, num_neg_silhouette_scores, min_silhouette_score,dev_from_mean_num_zone = plot_silhouette_scores(X_train_normalized,labels,metric,method,clustering_type)
            agglomerative_result_comb.append([silhouette_avg,num_neg_silhouette_scores, min_silhouette_score,metric,method,num_clusters,dev_from_mean_num_zone ])


methods = ['complete','average']#['single','complete','average','weighted',]
metrics = ['mahalanobis','manhattan']
for metric in metrics:
    for method in methods:
        for num_clusters in num_clusters_list:
            model = AgglomerativeClustering(n_clusters=num_clusters, metric=metric, linkage=method)
            model.fit(X_train_normalized)
            labels = model.labels_
            silhouette_avg, num_neg_silhouette_scores, min_silhouette_score,dev_from_mean_num_zone = plot_silhouette_scores(X_train_normalized,labels,metric,method,clustering_type)
            agglomerative_result_comb.append([silhouette_avg,num_neg_silhouette_scores, min_silhouette_score,metric,method,num_clusters,dev_from_mean_num_zone ])

In [ ]:
"""
Shortlist Agglomerative clustering

"""

agglomerative_result_comb_df = pd.DataFrame(agglomerative_result_comb,columns=["silhouette_avg","num_neg_silhouette_scores", "min_silhouette_score","metric","method","num_clusters","dev_from_mean_num_zone"])

sorted_best_agglomerative_result_comb = agglomerative_result_comb_df.sort_values(by=['num_neg_silhouette_scores','dev_from_mean_num_zone','min_silhouette_score'],ascending=[True,True,False])

sorted_best_agglomerative_result_comb

In [ ]:
"""
SHORTLISTED aggolmeratic clustering parameters combination for stats features

"""

plt.close('all')

num_clusters_list = [sorted_best_agglomerative_result_comb.iloc[0]['num_clusters']]
clustering_type = 'Agglomerative'


plot_title = 'TS statistical features'
mother = 'rbio2.2'
limit_level = 2
trend_angular_labels = ["Average Demand",'no_mean_crossing', "n95 demand", "Zero demands", "skewness"]

methods =  [sorted_best_agglomerative_result_comb.iloc[0]['method']] #['complete']#['single','complete','average','weighted',]
metrics = [sorted_best_agglomerative_result_comb.iloc[0]['metric']]
for metric,method,num_clusters in zip(metrics,methods,num_clusters_list):
    print('\n',metric,method,num_clusters)
    plot_title = 'TS stats features, '+ ', metric: '+metric+', method: '+', num_clusters: '+str(num_clusters)+'. '+', Clustering_type : '+clustering_type
    agglomerative_model = AgglomerativeClustering(n_clusters=num_clusters, metric=metric, linkage=method)
    agglomerative_model.fit(X_train_normalized)
    labels = agglomerative_model.labels_
    plot_silhouette_scores(X_train_normalized,agglomerative_model.labels_,metric,method,clustering_type)
    clustering_result_dict,all_cluster_feat_mins,all_cluster_feat_maxs = plot_stations_of_clusters(agglomerative_model.labels_, zone_met_10m_outflow_df,'met_demand',X_train_normalized,X_train_inv_normalized,'par_coords',trend_angular_labels,plot_title,mother,limit_level)





In [ ]:
"""
Visualize cluster representatives from every_days_avg_dem Time series statistical features using parcoords plot

"""

# #Add dummy outliers
# all_cluster_feat_mins = np.append( np.random.randint(0,1,(5,)) , all_cluster_feat_mins)
# all_cluster_feat_maxs = np.append( np.random.randint(0,1,(5,)) , all_cluster_feat_maxs)

plot_cluster_representatives(X_train_normalized, all_cluster_feat_mins, all_cluster_feat_maxs,'par_coords',trend_angular_labels)

In [ ]:
"""
Rank cluster from "highest demand" to "lowest demand" vaguely without the outlier.
"""
num_clusters = len(np.unique(agglomerative_model.labels_))
num_features = X_train_normalized.shape[1]

ranked_clusters = rank_clusterings(all_cluster_feat_mins,all_cluster_feat_maxs,num_clusters,num_features)
ranked_clusters

In [ ]:
len(all_cluster_feat_mins)

In [ ]:
"""
Display different ranked clusters on the same map
"""

colormap = plt.cm.RdYlGn  # Diverging red to green colormap
norm = mcolors.Normalize(vmin=min(ranked_clusters.keys()), vmax=max(ranked_clusters.keys()))

map_center = [40.7831, -73.9712]  # Manhattan coordinates (latitude, longitude)
map_zoom = 12  # Adjust the zoom level as needed
my_map = folium.Map(location=map_center, zoom_start=map_zoom,width='70%', height='70%')

marker_color = [ 'black','darkred','red','lightred','lightgreen','green','darkgreen',    'lightblue',    'blue',    'darkblue',]

for cluster in clustering_result_dict.keys(): # ourliers included
  stations = clustering_result_dict[cluster]
  color = mcolors.to_hex(colormap(norm(cluster)))
  for station in stations:
    folium.Marker(location=zone_to_gps_dict[int(station)], popup=str(station)+': '+color+'-'+str(cluster), icon=folium.Icon(color=marker_color[cluster+1]),tooltip=str(cluster)).add_to(my_map)
display(my_map)

In [ ]:
"""
Identify kmeans clustering parameter combination with best silhouette score for trend and seasonality stat features

"""

metrics = ['euclidean']
methods = ['single','complete','average','ward']
clustering_type = 'K-Means'

kmeans_result_comb = []

for num_cluster in range(4,8,1):
    for metric in metrics:
        for method in methods:
            km = TimeSeriesKMeans(n_clusters=num_cluster, metric=metric)
            km.fit(X_train_normalized)
            #plot_stations_of_clusters(km.labels_, Dim_Red_unmet_60min_outflow_week_df,'unmet_demand')
            silhouette_avg, num_neg_silhouette_scores, min_silhouette_score,dev_from_mean_num_zone = plot_silhouette_scores(X_train_normalized,km.labels_,metric,method,clustering_type)
            kmeans_result_comb.append([silhouette_avg,num_neg_silhouette_scores, min_silhouette_score,metric,method,num_cluster,dev_from_mean_num_zone ])


In [ ]:
"""
Shortlist kmeans clustering for TS trend stats features

"""

kmeans_result_comb_df = pd.DataFrame(kmeans_result_comb,columns=["silhouette_avg","num_neg_silhouette_scores", "min_silhouette_score","metric","method","num_clusters","dev_from_mean_num_zone"])

sorted_best_kmeans_result_comb = kmeans_result_comb_df.sort_values(by=['num_neg_silhouette_scores','dev_from_mean_num_zone','min_silhouette_score'],ascending=[True,True,False])

sorted_best_kmeans_result_comb

In [ ]:
"""
SHORTLISTED K-means clustering parameters combination for stats features

"""

plt.close('all')

plot_title = 'TS statistical features'
mother = 'rbio2.2'
limit_level = 2
trend_angular_labels = ["Average Demand",'no_mean_crossing', "n95 demand", "Zero demands", "skewness"]

metrics = [sorted_best_kmeans_result_comb.iloc[0]['metric']]
methods = [sorted_best_kmeans_result_comb.iloc[0]['method']]
clustering_type = 'K-Means'
num_clusters_list = [sorted_best_kmeans_result_comb.iloc[0]['num_clusters']]

for num_cluster in num_clusters_list:
    for metric in metrics:
        for method in methods:
            kmeans_model = TimeSeriesKMeans(n_clusters=num_cluster, metric=metric)
            kmeans_model.fit(X_train_normalized)
            silhouette_avg, num_neg_silhouette_scores, min_silhouette_score,dev_from_mean_num_zone = plot_silhouette_scores(X_train_normalized,kmeans_model.labels_,metric,method,clustering_type)
            clustering_result_dict,all_cluster_feat_mins,all_cluster_feat_maxs = plot_stations_of_clusters(kmeans_model.labels_, zone_met_10m_outflow_df,'met_demand',X_train_normalized,X_train_inv_normalized,'par_coords',trend_angular_labels,plot_title,mother,limit_level)




In [ ]:
"""
Visualize cluster representatives from every_days_avg_dem Time series statistical features using parcoords plot

"""

#Add dummy outliers
# all_cluster_feat_mins = np.append( np.random.randint(0,1,(5,)) , all_cluster_feat_mins)
# all_cluster_feat_maxs = np.append( np.random.randint(0,1,(5,)) , all_cluster_feat_maxs)

plot_cluster_representatives(X_train_normalized, all_cluster_feat_mins, all_cluster_feat_maxs,'par_coords',trend_angular_labels)

In [ ]:
"""
Rank cluster from "highest demand" to "lowest demand" vaguely without the outlier.
"""
num_clusters = len(np.unique(kmeans_model.labels_))
num_features = X_train_normalized.shape[1]

ranked_clusters = rank_clusterings(all_cluster_feat_mins,all_cluster_feat_maxs,num_clusters,num_features) # exclude outliers
ranked_clusters

In [ ]:
"""
Display different ranked clusters on the same map
"""

colormap = plt.cm.RdYlGn  # Diverging red to green colormap
norm = mcolors.Normalize(vmin=min(ranked_clusters.keys()), vmax=max(ranked_clusters.keys()))

map_center = [40.7831, -73.9712]  # Manhattan coordinates (latitude, longitude)
map_zoom = 12  # Adjust the zoom level as needed
my_map = folium.Map(location=map_center, zoom_start=map_zoom,width='70%', height='70%')

marker_color = [ 'black','darkred','red','lightred','lightgreen','green','darkgreen',    'lightblue',    'blue',    'darkblue',]

for cluster in clustering_result_dict.keys(): # ourliers included
  stations = clustering_result_dict[cluster]
  color = mcolors.to_hex(colormap(norm(cluster)))
  for station in stations:
    folium.Marker(location=zone_to_gps_dict[int(station)], popup=str(station)+': '+color+'-'+str(cluster), icon=folium.Icon(color=marker_color[cluster+1]),tooltip=str(cluster)).add_to(my_map)
display(my_map)

In [ ]:
"""
Decide which is the best clustering option/method out of the 3 clustering methods

"""

## Best results from each clustering

# hdbscan_best_score_comb_df = pd.DataFrame( {'dbcv':hdbscan_best_score_comb[-1][0] ,'num_outliers':hdbscan_best_score_comb[-1][1][5],'num_clusters': hdbscan_best_score_comb[-1][1][0], \
#                                             'min_cluster_size': hdbscan_best_score_comb[-1][1][1], 'min_samples': hdbscan_best_score_comb[-1][1][2], \
#                                             'cluster_selection_method': hdbscan_best_score_comb[-1][1][3], 'metric': hdbscan_best_score_comb[-1][1][4],},index=range(1)).transpose()

# sorted_best_agglomerative_result_comb_df = pd.DataFrame(sorted_best_agglomerative_result_comb.iloc[0])
# sorted_best_kmeans_result_comb_df = pd.DataFrame(sorted_best_kmeans_result_comb.iloc[0])

best_clustering_comb = pd.DataFrame( {'dbcv/silh_avg':[hdbscan_best_score_comb[-1][0],sorted_best_agglomerative_result_comb.iloc[0]['silhouette_avg'],sorted_best_kmeans_result_comb.iloc[0]['silhouette_avg'] ], \
                                      'num_outliers/neg_silh_score':[hdbscan_best_score_comb[-1][1][5],sorted_best_agglomerative_result_comb.iloc[0]['num_neg_silhouette_scores'],sorted_best_kmeans_result_comb.iloc[0]['num_neg_silhouette_scores'] ], \
                                      'min_silhouette_score':[np.NaN,sorted_best_agglomerative_result_comb.iloc[0]['min_silhouette_score'],sorted_best_kmeans_result_comb.iloc[0]['min_silhouette_score']],
                                      'num_clusters': [hdbscan_best_score_comb[-1][1][0],sorted_best_agglomerative_result_comb.iloc[0]['num_clusters'],sorted_best_kmeans_result_comb.iloc[0]['num_clusters'] ], \
                                      'metric': [hdbscan_best_score_comb[-1][1][4],sorted_best_agglomerative_result_comb.iloc[0]['metric'], sorted_best_kmeans_result_comb.iloc[0]['metric']], \
                                      },index=['hdbscan','agglomerative','kmeans'])

#TODO: shortlist clustering results based on hyperparamter conditions

model_labels = {'hdbscan':hdbscan_model.labels_,'agglomerative':agglomerative_model.labels_,'kmeans':kmeans_model.labels_}

sorted_best_clustering_comb = best_clustering_comb.sort_values(['num_outliers/neg_silh_score'],ascending=[True])
chosen_model = sorted_best_clustering_comb.index[0] # select the clustering method with minimum number of outliers.

sorted_best_clustering_comb


In [ ]:
"""
save final cluster labels results
"""
final_cluster_labels_df = pd.DataFrame(data={manhat_demand.index[-1]:model_labels[chosen_model]},index=range(0,len(model_labels[chosen_model]),1))
final_cluster_labels_df.to_csv('Manhattan_final_cluster_result_'+ str(manhat_demand.index[-1])+'.csv')
